# Кластеризация данных

## Импорт

In [ ]:
# импортируем необходимые модули
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy import integrate, optimize
from sklearn import preprocessing, model_selection, metrics
from sklearn.cluster import KMeans
# from kneed import KneeLocator - у меня не работает 
from IPython.display import display
%matplotlib inline


In [ ]:
# загружаем данные из файла в формате CSV, указываем символ-разделитель и сохраняем данные в дата-фрейм (переменную df)
df = pd.read_csv('../data/marketing_campaign.csv',sep='\t')
# проверяем, что данные загрузились - выводим первые несколько строк таблицы
df.head()

In [ ]:
# Raw features (before feature engineering)
num_features = [
    "Year_Birth", "Income", "Recency",
    "MntWines", "MntFruits", "MntMeatProducts", "MntFishProducts", "MntSweetProducts", "MntGoldProds",
    "NumDealsPurchases", "NumWebPurchases", "NumCatalogPurchases", "NumStorePurchases", "NumWebVisitsMonth"
]

cat_features = [
    "Education", "Marital_Status", "Response", "Complain",
    "Kidhome", "Teenhome",
    "AcceptedCmp1", "AcceptedCmp2", "AcceptedCmp3", "AcceptedCmp4", "AcceptedCmp5"
]

# --- Category normalization ---
df["Education"] = df["Education"].replace({"2n Cycle": "Pre-Graduate", "Basic": "Pre-Graduate"})

df["Marital_Status"] = df["Marital_Status"].replace({
    "Married": "Married/Together",
    "Together": "Married/Together",
    "Single": "Single",
    "Divorced": "Other",
    "Widow": "Other",
    "Alone": "Other",
    "Absurd": "Other",
    "YOLO": "Other"
})

# --- Feature engineering ---
df["Kids"] = df["Kidhome"].astype("int8") + df["Teenhome"].astype("int8")

df["Expenses"] = (
    df["MntWines"] + df["MntFruits"] + df["MntMeatProducts"] +
    df["MntFishProducts"] + df["MntSweetProducts"] + df["MntGoldProds"]
)

df["TotalAcceptedCmp"] = (
    df["AcceptedCmp1"].astype("int8") + df["AcceptedCmp2"].astype("int8") +
    df["AcceptedCmp3"].astype("int8") + df["AcceptedCmp4"].astype("int8") +
    df["AcceptedCmp5"].astype("int8")
)

df["TotalNumPurchases"] = (
    df["NumWebPurchases"] + df["NumCatalogPurchases"] +
    df["NumStorePurchases"] + df["NumDealsPurchases"]
)

# Drop columns that were aggregated into engineered features
df.drop(
    columns=[
        "Kidhome", "Teenhome",
        "MntWines", "MntFruits", "MntMeatProducts", "MntFishProducts", "MntSweetProducts", "MntGoldProds",
        "AcceptedCmp1", "AcceptedCmp2", "AcceptedCmp3", "AcceptedCmp4", "AcceptedCmp5",
        "NumWebPurchases", "NumCatalogPurchases", "NumStorePurchases", "NumDealsPurchases",
    ],
    inplace=True
)

# Post-processing / binning
df["Kids"] = df["Kids"].replace({0: "No Kid", 1: "Has Kids", 2: "Has Kids", 3: "Has Kids"})
df["TotalAcceptedCmp"] = df["TotalAcceptedCmp"].replace({0: "0", 1: ">0", 2: ">0", 3: ">0", 4: ">0"})

# Final feature lists (after feature engineering)
num_features = ["Income", "Recency", "NumWebVisitsMonth", "Expenses", "TotalNumPurchases"]
cat_features = ["Education", "Marital_Status", "Response", "Complain", "Kids", "TotalAcceptedCmp"]

In [ ]:
def check_skewness(data = df, numerical_features = num_features, highlight=True, sort=True):
    skewness_dict = {}
    skew_feature = []
    for feature in numerical_features:
        skew = data[feature].skew(skipna=True)
        skewness_dict[feature] = skew

    skew_df = pd.DataFrame.from_dict(skewness_dict, orient="index", columns=["Skewness"])
    if sort:
        skew_df = skew_df.reindex(skew_df["Skewness"].abs().sort_values(ascending=False).index)
    else:
        pass
    
    print(f"\n🔍 Skewness for dataset:")
    print("-"*70)
    print(f"{'Feature':<30} | {'Skewness':<10} | {'Remark'}")
    print("-"*70)
    for feature, row in skew_df.iterrows():
        skew = row["Skewness"]
        abs_skew = abs(skew)
        if abs_skew > 1:
            remark = "Highly skewed"
            color = "\033[91m"  
        elif abs_skew > 0.5:
            remark = "Moderately skewed"
            color = "\033[93m"  
        else:
            remark = "Approximately symmetric"
            color = ""
        endc = "\033[0m" if color else ""
        if highlight and color:
            print(f"{color}{feature:<30} | {skew:>+10f} | {remark}{endc}")
            skew_feature.append(feature)
        else:
            print(f"{feature:<30} | {skew:>+10f} | {remark}")
    print("-"*70)
    return skew_feature, skew_df

skew_feature, skew_df = check_skewness()

In [ ]:
skew_feature, skew_df = check_skewness(data=df, numerical_features=num_features)